In [15]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

import nltk
import regex as re
# import requests
# import ast
# from bs4 import BeautifulSoup as bsoup

#so I can get sounds to play
from IPython.display import Audio
wave = np.sin(2*np.pi*400*np.arange(10000*2)/10000)
#So I can see multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [298]:
df = pd.read_csv('recipe_output_new.csv', names = range(80), sep = ",", quotechar = '"', skipinitialspace=True, error_bad_lines = False, dtype=object, index_col=False)
df.rename(columns={0: 'Link', 1: 'Title', 2: 'TotalTime'}, inplace=True)

In [333]:
len(df)
df.tail(5)

133648

,Website,Link,Title,TotalTime,3,4,5,6,7,8,...,70,71,72,73,74,75,76,77,78,79
133661,bonappetit,https://www.bonappetit.com/recipe/watermelon-g...,Watermelon Gazpacho with Feta Crema,0,"1 pound seedless watermelon, rind removed, coa...","1 large beefsteak tomato, coarsely chopped","1 English hothouse cucumber, peeled, coarsely ...","1 jalapeño, seeds removed, sliced",2 tablespoons olive oil,2 tablespoons Sherry vinegar or red wine vinegar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133662,epicurious,https://www.epicurious.com/recipes/food/views/...,Barbecue Pork Kebabs With Blistered-Chile–Pump...,0,1/4 large white onion,2 jalapeños,1/4 cup chopped cilantro,"1/4 cup finely chopped unsalted, roasted pumpk...",1/4 cup olive oil,3 tablespoons fresh lime juice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133663,epicurious,https://www.epicurious.com/recipes/food/views/...,Brochette de Lapin aux Pruneaux,0,"1 rabbit, divided into 2 legs, 2 loins, and 2 ...","4 slices bacon, cut into 3 pieces each",12 pitted prunes,"1 red onion, cut into 12 pieces",2 tablespoons olive oil,1 teaspoon salt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133664,epicurious,https://www.epicurious.com/recipes/food/views/...,Patatine e Carciofi Arrosto roasted Potatoes a...,0,1 tablespoon finely grated fresh lemon zest,3 tablespoons finely chopped fresh flat-leafed...,2 teaspoons minced garlic,2 pounds red potatoes (about 1 1/2 inches in d...,6 tablespoons extra-virgin olive oil,freshly ground black pepper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133665,epicurious,https://www.epicurious.com/recipes/food/views/...,Salmon Cakes With Arugula Salad,0,"20 ounces canned skinless, boneless pink salmon",1/4 cup whole-wheat panko breadcrumbs,"2 tbsp plus 1 1/2 tsp grainy mustard, divided",2 tbsp light mayonnaise,"1½ tbsp chopped dill, divided",1 tbsp plus 1/2 teaspoon finely chopped shallot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
df.loc[:,'Website'] = pd.np.nan

In [301]:
#Move website to front
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [302]:
p1 = re.compile("(://www.)+?([^.]+)")
p2 = re.compile("(://)+?([^.]+)")
text1 = 'https://www.bonappetit.com/recipe/strawberry-i...'
text2 = 'https://tastykitchen.com/recipes/appetizers-an...'
    
def f(x):
    s =  p1.search(x)
    if pd.isnull(s):
        s =  p2.search(x)
        if pd.isnull(s):
            return pd.np.nan
        else:
            return s.group(2)
    else:
        return s.group(2)

df.loc[:,'Website'] = df['Link'].map(f)

In [330]:
df.iloc[:,0:4].isna().sum()
df[df.Website.isna()]

Website        0
Link           0
Title        976
TotalTime      1
dtype: int64

,Website,Link,Title,TotalTime,3,4,5,6,7,8,...,70,71,72,73,74,75,76,77,78,79


In [329]:
# num = 73190
# # df.iloc[num-1:num+2,:5]
# new = pd.Series([pd.np.nan])
# new = new.append(df.iloc[(num),:-1])
# df.append(new, ignore_index=True)
df.drop(index = [ 73190,  73191,  73192,  73193,  73194,  73195,  73196,  73388, 73389,  73390,  73391,  73392,  73393,  73394,  73395,  90028, 123403, 123404], inplace = True)
# df = df.reset_index(drop=True)
# df.iloc[num-1:num+2,:5]
# df[df.Website.isna()]
# df.append(df.iloc[[ 73190,  73191,  73192,  73193,  73194,  73195,  73196,  73388,
#              73389,  73390,  73391,  73392,  73393,  73394,  73395,  90028,
#             123403, 123404]].shift(1, axis=1))


In [ ]:
num = 9931
df.iloc[(num-4):(num+3),0:18]
# df[df["Recipe"].str.contains('Hake')]
df.iloc[num-4]
# df.loc[num]

num = 5459
df.iloc[num-1:num+2,:5]
new = pd.Series([pd.np.nan, 'https://www.bonappetit.com/recipe/brined-and-roasted-rosemary-chile-almonds',"Brined and Roasted Rosemary-Chile Almonds"])
new = new.append(df.iloc[(num),2:-1])
df.append(new, ignore_index=True)
df.drop(index = num, inplace = True)
df = df.reset_index(drop=True)
df[df.Website.isna()]

num = 9069
df.iloc[num-1:num+2,:5]
new = pd.Series(['bonappetit', 'https://www.bonappetit.com/recipe/noodle-salad-with-chicken-and-chile-scallion-oil',"Noodle Salad with Chicken and Chile-Scallion Oil",0])
new = new.append(df.iloc[(num),2:-4])
new = new[~new.index.duplicated()]
df.append(new, ignore_index=True)
df.drop(index = num, inplace = True)
df = df.reset_index(drop=True)
df[df.Website.isna()]

num = 9260
df.iloc[num-1:num+2,:5]
new = pd.Series(['bonappetit', 'https://www.bonappetit.com/recipe/pan-seared-hake-with-kale-and-chorizo',"Pan-Seared Hake with Kale and Chorizo",0], name = range(80))
new = new.append(df.iloc[(num),3:-4])
new = new[~new.index.duplicated()]
df.append(new, ignore_index=True)
df.drop(index = (num), inplace = True)
df = df.reset_index(drop=True)
df[df.Website.isna()]

num = 9928
df.iloc[num-1:num+2,:5]
new = pd.Series(['bonappetit', 'https://www.bonappetit.com/recipe/radicchio-fennel-and-olive-panzanella',"Radicchio, Fennel, and Olive Panzanella",0], name = range(80))
new = new.append(df.iloc[(num-2),3:-4])
new = new[~new.index.duplicated()]
df.append(new, ignore_index=True)
df.drop(index = (num), inplace = True)
df = df.reset_index(drop=True)
df[df.Website.isna()]

num = 10559
df.iloc[num-1:num+2,:5]
new = pd.Series(['bonappetit', 'https://www.bonappetit.com/recipe/sausage-and-broccoli-rabe-frittata',"Sausage and Broccoli Rabe Frittata",0])
new = new.append(df.iloc[(num),3:-4])
new = new[~new.index.duplicated()]
df.append(new, ignore_index=True)
df.drop(index = (num), inplace = True)
df = df.reset_index(drop=True)
df[df.Website.isna()]

num = 10748
df.iloc[num-1:num+2,:5]
new = pd.Series(['bonappetit', 'https://www.bonappetit.com/recipe/shaved-honeydew-fennel-and-olive-salad',"Shaved Honeydew, Fennel, and Olive Salad",0])
new = new.append(df.iloc[(num),3:-4])
new = new[~new.index.duplicated()]
df.append(new, ignore_index=True)
df.drop(index = (num), inplace = True)
df = df.reset_index(drop=True)
df[df.Website.isna()]

num = 11193
df.iloc[num-1:num+2,:5]
new = pd.Series(['bonappetit', 'https://www.bonappetit.com/recipe/spiced-fava-bean-soup-with-rice-and-tomato',"Spiced Fava Bean Soup with Rice and Tomato",0])
new = new.append(df.iloc[(num),3:-4])
new = new[~new.index.duplicated()]
df.append(new, ignore_index=True)
df.drop(index = (num), inplace = True)
df = df.reset_index(drop=True)
df[df.Website.isna()]

In [334]:
df.to_csv(path_or_buf = f'recipe_overclean.csv', header = False, index = False)

In [ ]:
#concat text by rows grouped by 
ingredients = df.groupby('Website').agg(lambda x: ', '.join(x.astype(str)))
ingredients = ingredients[['Title', 4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79]]

In [ ]:
ingredients.loc[:,'AggIngredients'] = ingredients[[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79]].astype(str).apply(' '.join, axis=1)


In [ ]:
ingredients = ingredients[['Title','AggIngredients']]

In [ ]:
#Cleanup separate commas, all lowercase, stem select words
ingredients.Title = ingredients.Title.str.replace(',',' , ')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace(',',' , ')
ingredients.Title = ingredients.Title.str.replace('(',' ')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('(',' ')
ingredients.Title = ingredients.Title.str.replace(')',' ')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace(')',' ')

ingredients.Title = ingredients.Title.str.lower()
ingredients.AggIngredients = ingredients.AggIngredients.str.lower()

ingredients.Title = ingredients.Title.str.replace('nan',' ')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('nan',' ')

ingredients.Title = ingredients.Title.str.replace('potatoes','potato')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('potatoes','potato')
ingredients.Title = ingredients.Title.str.replace('potatoes','potato')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('tomatoes','tomato')
ingredients.Title = ingredients.Title.str.replace('eggs','egg')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('eggs','egg')
#regex? clove * garlic, garlic clove
ingredients.Title = ingredients.Title.str.replace('garlic clove','garlic')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('garlic clove','garlic')
ingredients.Title = ingredients.Title.str.replace('cloves of garlic','garlic')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('cloves of garlic','garlic')
ingredients.Title = ingredients.Title.str.replace('clove of garlic','garlic')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('clove of garlic','garlic')

#!?

In [3]:
#Read in 
ingredients = pd.read_csv('ingredients_agg.csv', names = ['Title','AggIngredients'], sep = ",", quotechar = '"', skipinitialspace=True, error_bad_lines = False, dtype=object)

In [4]:
ingredients.Title = ingredients.Title.str.replace('extra virgin','extra-virgin')
ingredients.AggIngredients = ingredients.AggIngredients.str.replace('extra virgin','extra-virgin')

In [5]:
ingredients.to_csv(path_or_buf = f'ingredients_agg.csv', header = False)

In [3]:
df = pd.read_csv('recipe_overclean.csv', sep = ",", quotechar = '"', dtype=object, index_col=0)
df["TotalIng"] = pd.to_numeric(df["TotalIng"])#This actually is num of nan

#df.to_csv(path_or_buf = f'recipe_overclean.csv', header = False, index = False)

In [7]:
df.dtypes


Website      object
Link         object
Title        object
TotalTime    object
4            object
5            object
6            object
7            object
8            object
9            object
10           object
11           object
12           object
13           object
14           object
15           object
16           object
17           object
18           object
19           object
20           object
21           object
22           object
23           object
24           object
25           object
26           object
27           object
28           object
29           object
              ...  
51           object
52           object
53           object
54           object
55           object
56           object
57           object
58           object
59           object
60           object
61           object
62           object
63           object
64           object
65           object
66           object
67           object
68           object
69           object


In [9]:
#View some summary data by website
agg1 = df.groupby("Website").size()
agg2 = 76 - df.groupby('Website')[["TotalIng"]].mean() #To convert to not nan
website = pd.concat([agg1, agg2], axis=1, sort=False)
website.rename(columns={0: 'NumRecipes', 'TotalIng': 'AveNumIngr'}, inplace=True)

In [10]:
website

,NumRecipes,AveNumIngr
Website,,
100daysofrealfood,8,14.250000
101cookbooks,15,13.200000
101cookingfortwo,5,10.600000
12tomatoes,40,16.075000
196flavors,3,18.333333
1mrecipes,2,15.000000
247lowcarbdiner,1,11.000000
28cooks,1,14.000000
2stews,2,27.000000


In [11]:
#Read in
ingredients = pd.read_csv('ingredients_agg.csv', names = ['Title','AggIngredients'], sep = ",", quotechar = '"', skipinitialspace=True, error_bad_lines = False, dtype=object)

In [ ]:
#summary statistic,  word frequency
wordfreq_Title = ingredients.Title.apply(lambda x: pd.value_counts(str(x).split(" "))).sum(axis = 0)
wordfreq_Title = wordfreq_Title[3:]

In [13]:
type(wordfreq_Title)

pandas.core.series.Series

In [17]:
pickling_on = open("wordfreq_Title.pickle","wb")
pickle.dump(wordfreq_Title, pickling_on)
pickling_on.close()

In [18]:
wordfreq_AggIng = ingredients.AggIngredients.apply(lambda x: pd.value_counts(str(x).split(" "))).sum(axis = 0)
wordfreq_AggIng = wordfreq_AggIng[4:]

In [19]:
pickling_on = open("wordfreq_AggIng.pickle","wb")
pickle.dump(wordfreq_AggIng, pickling_on)
pickling_on.close()